# Company Valuation Notebook

This Jupyter Notebook is designed to perform a company valuation based on a set of financial inputs and assumptions. The notebook calculates various financial metrics and ultimately provides the value per share of the company. It is an extension of Damodaran's fcffsimpleginzu spreadsheet. As of now, only default assumptions are at play and key metrics like cost of capital needs to be entered manually. I will try to a provision of that too in the future

## Inputs
The inputs required for this valuation are provided in the below section of the notebook. These inputs include:

- Revenue growth rate for year 1 (`rev_gr_1`)
- Compound Annual Growth Rate (CAGR) for years 2-5 (`cagr_2_5`)
- Risk-free rate (`risk_free_rate`)
- Revenue for the last 12 months (`rev_r12m`)
- EBIT for the last 12 months (`ebit_r12m`)
- R&D amortization years (`rnd_amr_yrs`)
- Current R&D (`curr_rnd`)
- Previous R&D (`prev_rnd`)
- Target Pre-tax Operating margin (`tgt_ebit_margin`)
- Years to convert to target margin (`yr_conv_margin`)
- Effective tax rate (`eff_tx_rate`)
- Marginal tax rate (`mar_tax_rate`)
- Sales to capital ratio for years 1-5 (`sales_to_cap_1_5`)
- Sales to capital ratio for years 6-10 (`sales_to_cap_6_10`)
- Mature market Equity Risk Premium (ERP) (`mature_market_erp`)
- Country ERP (`country_risk_premium`)
- Cost of capital (`cost_of_capital`)
- Probability of failure (`prob_failure`)
- Book value of debt (`book_val_debt`)
- Cash and marketable securities (`cash`)
- Non-operating assets (`non_op_assets`)
- Minority interests (`minority_interests`)
- Shares outstanding (`shares_out`)

## Description
This notebook performs the following calculations:

1. **Growth Rates**: Calculates the growth rates for the next 10 years.
2. **Revenues**: Projects the revenues for the next 10 years based on the growth rates.
3. **Amortization Weights**: Calculates the amortization weights for R&D expenses.
4. **Value of Research Asset**: Computes the value of the research asset.
5. **Adjusted EBIT**: Adjusts the EBIT for R&D expenses.
6. **Operating Margins**: Projects the operating margins for the next 10 years.
7. **Operating Income**: Calculates the operating income (EBIT) for the next 10 years.
8. **Tax Rates**: Projects the tax rates for the next 10 years.
9. **EBIT(1-tax)**: Calculates the after-tax EBIT for the next 10 years.
10. **Reinvestment**: Computes the reinvestment required for the next 10 years.
11. **Free Cash Flow to Firm (FCFF)**: Calculates the FCFF for the next 10 years.
12. **Cost of Capital**: Projects the cost of capital for the next 10 years.
13. **Discount Factors**: Computes the discount factors for the next 10 years.
14. **Present Value of FCFF**: Calculates the present value of FCFF for the next 10 years.
15. **Terminal Year Calculations**: Computes the terminal value and its present value.
16. **Valuation**: Calculates the value of operating assets, equity, and value per share.

## Default assumptions
1. In stable growth, I will assume that your firm will have a cost of capital similar to that of typical mature companies
2. I will assume that your firm will earn a return on capital equal to its cost of capital after year 10. I am assuming that whatever competitive advantages you have today will fade over time.
3. I will assume that your firm has no chance of failure over the foreseeable future.
4. I assume that reinvestment in a year translates into growth in the next year, i.e., there is a one year lag between reinvesting and generating growth from that reinvestment.
5. I will assume that your effective tax rate will adjust to your marginal tax rate by your terminal year. If you override this assumption, I will leave the tax rate at your effective tax rate.
6. I will assume that you have no losses carried forward from prior years ( NOL) coming into the valuation. If you have a money losing company, you may want to override this.
7. I will asssume that today's risk free rate will prevail in perpetuity. If you override this assumption, I will change the riskfree rate after year 10. (**Override feature unavailable as of now**)
8. I will assume that the growth rate in perpetuity will be equal to the risk free rate. This allows for both valuation consistency and prevents "impossible" growth rates.
9. I have assumed that none of the cash is trapped (in foreign countries) and that there is no additional tax liability coming due and that cash is a neutral asset.



In [49]:
# Inputs
rev_gr_1 = 12/100  # Revenue growth rate for year 1
cagr_2_5 = 12/100   # CAGR for years 2-5
risk_free_rate = 4.08/100 # Risk free rate
rev_r12m = 574785 # Revenue for the last 12 months (in millions)
ebit_r12m = 36852 # EBIT for the last 12 months (in millions)
rnd_amr_yrs = 3 # R&D amortization years
curr_rnd = 85622 # Current R&D (in millions)
prev_rnd = [73213, 56052, 42740, 0, 0, 0, 0, 0, 0, 0] # Previous R&D (in millions)
#ebit_margin_1 = 11.33/100 # Operating margin for year 1 (Calculated below in Operating margin section)
tgt_ebit_margin = 14/100 # Target Pre-tax Operating margin
yr_conv_margin = 5 # Years to convert to target margin
eff_tx_rate = 19/100 # Effective tax rate
mar_tax_rate = 25/100 # Marginal tax rate
sales_to_cap_1_5 = 1.5 # Sales to capital ratio Years 1-5
sales_to_cap_6_10 = 1.5 # Sales to capital ratio Years 6-10
mature_market_erp = 4.11/100 # Mature market ERP (Damodaran)
country_risk_premium = 0 # Country ERP (For US)
cost_of_capital = 8.6/100 # Cost of capital
prob_failure = 0 # Probability of failure
book_val_debt = 161574 # Book value of debt (in millions)
cash = 86780 # Cash and marketable securities (in millions)
non_op_assets = 2954 # Cross holdings and other Non-operating assets (in millions)
minority_interests = 0 # Minority interests (in millions)
shares_out = 10492 # Shares outstanding (in millions)

Growth rates list

In [50]:
# Create a list of 10 growth rates
growth_rates = [rev_gr_1]  # First year growth rate

# Year two to five will be cagr_2_5
growth_rates.extend([cagr_2_5] * 4)

# Calculate the decrement for years six to ten
decrement = (cagr_2_5 - risk_free_rate) / 5

# Year six to ten growth rates
for i in range(1, 6):
    growth_rates.append(cagr_2_5 - decrement * i)

print(growth_rates)

[0.12, 0.12, 0.12, 0.12, 0.12, 0.10416, 0.08832, 0.07247999999999999, 0.056639999999999996, 0.0408]


Revenues list

In [51]:
revenues = [rev_r12m * (1 + growth_rates[0])]  # Revenue for year 1

# Calculate revenues for subsequent years
for i in range(1, len(growth_rates)):
    revenues.append(revenues[-1] * (1 + growth_rates[i]))

print(revenues) 

[643759.2000000001, 721010.3040000001, 807531.5404800002, 904435.3253376003, 1012967.5643781124, 1118478.2658837365, 1217262.266326588, 1305489.4353899392, 1379432.3570104255, 1435713.1971764509]


Amortization weights

In [52]:
# Calculate the number of non-zero entries in prev_rnd
non_zero_entries = sum(1 for rnd in prev_rnd if rnd != 0)

# Calculate the amortization weights
amortization_weights = [i / non_zero_entries for i in range(non_zero_entries, 0, -1)]

print(amortization_weights)

[1.0, 0.6666666666666666, 0.3333333333333333]


In [53]:
# Value of research asset
val_rnd = sum(a * b for a, b in zip(([curr_rnd] + prev_rnd)[:len(amortization_weights)], amortization_weights))
val_rnd

153114.66666666666

In [54]:
#Amortization for this year
sum_prev_rnd = sum([rnd / rnd_amr_yrs for rnd in prev_rnd])
sum_prev_rnd

57335.0

In [55]:
# Adjustment to EBIT due to R&D
adj_ebit = curr_rnd - sum_prev_rnd
adj_ebit

28287.0

Adjusted operating income

In [56]:
# Adjusted EBIT (Base Year)
if curr_rnd != 0:
    ebit_adj = ebit_r12m + adj_ebit
else: 
    ebit_adj = ebit_r12m
ebit_adj

65139.0

Operating Margins

In [57]:
ebit_margin_0 = ebit_adj / rev_r12m
ebit_margin_1 = ebit_margin_0 # Assuming same margin for year 1

In [58]:
# Initialize the list with the operating margin for the first year
operating_margins = [ebit_margin_1]

# Calculate the step change for the operating margin
step_change = (tgt_ebit_margin - ebit_margin_1) / (yr_conv_margin)

# Calculate the operating margins for years 2 to the year of convergence
for i in range(1, yr_conv_margin):
    operating_margins.append(tgt_ebit_margin - step_change*(yr_conv_margin - i - 1))

# From the year of convergence onwards, the operating margin remains the same as the target one
operating_margins.extend([tgt_ebit_margin] * (10 - yr_conv_margin + 1))

print(operating_margins)

[0.11332759205616012, 0.12399655523369607, 0.12933103682246405, 0.13466551841123203, 0.14, 0.14, 0.14, 0.14, 0.14, 0.14, 0.14]


Operating Income

In [59]:
ebits = [revenue * margin for revenue, margin in zip(revenues, operating_margins)]
print(ebits)

[72955.68000000001, 89402.793984, 104438.89139712002, 121796.25195601924, 141815.45901293575, 156586.95722372312, 170416.71728572235, 182768.5209545915, 193120.5299814596, 200999.84760470313]


In [60]:
# Initialize the list with the effective tax rate for the first 5 years
tax_rates = [eff_tx_rate] * 5

# Calculate the step change for the tax rate
tax_rate_step_change = (mar_tax_rate - eff_tx_rate) / 5

# Calculate the tax rates for years 6 to 10
for i in range(1, 6):
    tax_rates.append(eff_tx_rate + tax_rate_step_change * i)

print(tax_rates)

[0.19, 0.19, 0.19, 0.19, 0.19, 0.202, 0.214, 0.226, 0.238, 0.25]


EBIT(1-tax)

In [61]:
ebit_minus_tax = [ebit * (1 - tax_rate) for ebit, tax_rate in zip(ebits, tax_rates)]
print(ebit_minus_tax)

[59094.10080000001, 72416.26312704, 84595.50203166723, 98654.96408437559, 114870.52180047797, 124956.39186453106, 133947.53978657778, 141462.83521885384, 147157.8438458722, 150749.88570352734]


Reinvestment

In [62]:
# Initialize the list of reinvestment
reinvestment = []

# Calculate reinvestment for years 1 to 5 using sales_to_cap_1_5
for i in range(5):
    reinvestment.append((revenues[i + 1] - revenues[i]) / sales_to_cap_1_5)

# Calculate reinvestment for years 6 to 9 using sales_to_cap_6_10
for i in range(5, 9):
    reinvestment.append((revenues[i + 1] - revenues[i]) / sales_to_cap_6_10)

# Calculate terminal year revenue
terminal_year_revenue = revenues[9] * (1 + risk_free_rate)

# Calculate reinvestment for year 10
reinvestment.append((terminal_year_revenue - revenues[9]) / sales_to_cap_6_10)

print(reinvestment)

[51500.73600000003, 57680.82432000005, 64602.52323840008, 72354.82602700808, 70340.46767041607, 65856.00029523438, 58818.11270890074, 49295.2810803242, 37520.56011068355, 39051.39896319946]


FCFF

In [63]:
fcff = [ebit - reinvest for ebit, reinvest in zip(ebit_minus_tax, reinvestment)]
print(fcff)

[7593.364799999974, 14735.438807039951, 19992.978793267153, 26300.138057367512, 44530.05413006191, 59100.391569296684, 75129.42707767704, 92167.55413852964, 109637.28373518866, 111698.48674032788]


Terminal cost of capital

In [64]:
terminal_cost_of_capital = risk_free_rate + mature_market_erp + country_risk_premium

In [65]:
# Initialize the list with the cost of capital for the first 5 years
cost_of_capital_list = [cost_of_capital] * 5

# Calculate the step change for the cost of capital
cost_of_capital_step_change = (terminal_cost_of_capital - cost_of_capital) / 5

# Calculate the cost of capital for years 6 to 10
for i in range(1, 6):
    cost_of_capital_list.append(cost_of_capital + cost_of_capital_step_change * i)

print(cost_of_capital_list)

[0.086, 0.086, 0.086, 0.086, 0.086, 0.08517999999999999, 0.08435999999999999, 0.08354, 0.08272, 0.0819]


In [66]:
# Initialize the list of discount factors with the first year's discount factor
discount_factors = [1 / (1 + cost_of_capital_list[0])]

# Calculate the discount factors for subsequent years
for i in range(1, 10):
    discount_factors.append(discount_factors[-1] / (1 + cost_of_capital_list[i]))

print(discount_factors)

[0.9208103130755064, 0.8478916326662121, 0.7807473597294771, 0.7189202207453748, 0.6619891535408607, 0.6100270494672412, 0.562568749739239, 0.5191951840626456, 0.4795285799307722, 0.44322819108122025]


Present value of FCFF

In [67]:
pv_fcff = [fcf * discount_factor for fcf, discount_factor in zip(fcff, discount_factors)]
print(pv_fcff)

[6992.048618784506, 12494.055268154165, 15609.465405970757, 18907.701057836483, 29478.412840688394, 36052.837491376675, 42265.4678597141, 47852.95023555775, 52574.21097700216, 49507.91822442519]


Terminal Year

In [68]:
terminal_ebit = terminal_year_revenue * tgt_ebit_margin
terminal_ebit_minus_tax = terminal_ebit * (1 - mar_tax_rate)
terminal_reinvestment = (risk_free_rate/terminal_cost_of_capital) * terminal_ebit_minus_tax
terminal_fcff = terminal_ebit_minus_tax - terminal_reinvestment
terminal_value = terminal_fcff / (terminal_cost_of_capital - risk_free_rate)
pv_terminal_value = terminal_value * discount_factors[-1]
pv_fcff_sum = sum(pv_fcff) 
sum_pv = pv_fcff_sum + pv_terminal_value
val_op_assets = sum_pv # Add Probability of failure assumption here in future
val_equity = val_op_assets - book_val_debt - minority_interests + cash + non_op_assets 
val_per_share = val_equity / shares_out
print(f"Terminal EBIT: {terminal_ebit:.2f}")
print(f"Terminal EBIT minus Tax: {terminal_ebit_minus_tax:.2f}")
print(f"Terminal Reinvestment: {terminal_reinvestment:.2f}")
print(f"Terminal FCFF: {terminal_fcff:.2f}")
print(f"Terminal Value: {terminal_value:.2f}")
print(f"Present Value of terminal Value: {pv_terminal_value:.2f}")
print(f"Sum of Present Value of FCFF: {pv_fcff_sum:.2f}")
print(f"Value of Operating Assets: {val_op_assets:.2f}")
print(f"Value of Equity: {val_equity:.2f}")
print(f"Value per Share: {val_per_share:.2f}")

Terminal EBIT: 209200.64
Terminal EBIT minus Tax: 156900.48
Terminal Reinvestment: 78162.88
Terminal FCFF: 78737.60
Terminal Value: 1915756.79
Present Value of terminal Value: 849117.42
Sum of Present Value of FCFF: 311735.07
Value of Operating Assets: 1160852.48
Value of Equity: 1089012.48
Value per Share: 103.79
